In [1]:
import numpy as np
import pandas as pd
import random
import networkx as nx
from IPython.display import Image
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn import metrics

In [2]:
GMissingEdges = nx.read_gml("GraphMissingEdges.gml")

# pos = nx.spring_layout(GMissingEdges)
# # nx.draw(GMissingEdges, cmap = plt.get_cmap('rainbow'), with_labels=True, pos=pos)

# n = GMissingEdges.number_of_nodes()
# m = GMissingEdges.number_of_edges()
# print("Number of nodes : %d" % n)
# print("Number of edges : %d" % m)
# print("Number of connected components : %d" % nx.number_connected_components(GMissingEdges))

In [3]:
# Remove 20% das arestas
proportion_edges = 0.2
edge_subset = random.sample(GMissingEdges.edges(), int(proportion_edges * GMissingEdges.number_of_edges()))

# Cria uma cópia do grafo e remove arestas
GMissingEdgesTrain = GMissingEdges.copy()
GMissingEdgesTrain.remove_edges_from(edge_subset)
# print(edge_subset)

In [4]:
dfToEvaluate = pd.DataFrame(edge_subset)
dfToEvaluate = dfToEvaluate.rename(columns={0:'venue1',1:'venue2'})
dfToEvaluate.to_csv("edgesToEvaluateTest.csv", index_label='linkID')
dfToEvaluate

,venue1,venue2
0,t4PZ3r2eTPeDXWNiV4Cr0g,UfNTlvEJCOQ7XDRj0LzEtQ
1,Qzbcq82RJKIcAl0HSoSBJQ,snw9iNNLpFYZeHotW00uVA
2,sjxNnjKwWn7H6QcsAhhXww,nk9lP-iXi7xklwN3VYf6Ew
3,4twpbw7n4DmsLxAm6-sMkg,NEYMAeK1BeR2E_vELnEIDA
4,aEZ_EoArhpjV2p0q5MXFMA,Nq0UHjD_cKW9UbIJplQf_g
...,...,...
10593,6f2imTsIysASgHh2t91OOQ,Jcyu0ml7rxizEA8giSH-8A
10594,lVTn2NO9-FGfxKjfleZUqw,rHKxy2FwlE8xxK9XOpigtQ
10595,5m2JukTajxMk-qS5W4s-FA,QWWg3mbz1VkBjfPr0gLRsQ
10596,Fgje0saJKZkCByBMj4SlhA,2lCjKqGK2s4FrJVPjJK1fg


In [5]:
# Fazendo previsões com o jaccard
pred_jaccard = list(nx.jaccard_coefficient(GMissingEdgesTrain))

score_jaccard = [s for (u,v,s) in pred_jaccard]
label_jaccard = [(u,v) in edge_subset for (u,v,s) in pred_jaccard]

# Computa o AUC score da curva ROC
fpr_jaccard, tpr_jaccard, _ = metrics.roc_curve(label_jaccard, score_jaccard)
auc_jaccard = roc_auc_score(label_jaccard, score_jaccard)
print(auc_jaccard)

In [ ]:
import matplotlib.pyplot as plt
plt.title('ROC')
plt.plot(fpr_jaccard, tpr_jaccard, 'b', label = 'AUC = %0.2f' % auc_jaccard)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('Taxa Verdadeiro Positivo')
plt.xlabel('Taxa de Falso Positivo')
plt.show()